<a href="https://colab.research.google.com/github/Jahan08/Amber-tutorial/blob/main/Retrieve-Data-Cancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install chembl_webresource_client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 KB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 KB 4.6 MB/s eta 0:00:00
  Attempting uninstall: attrs
    Found existing installation: attrs 22.2.0
    Uninstalling attrs-22.2.0:
      Successfully uninstalled attrs-22.2.0


In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

In [3]:
target = new_client.target
target_query = target.search('HEK293')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Homo sapiens,HEK293,20.0,False,CHEMBL614818,[],CELL-LINE,9606


In [4]:
selected_target = targets.target_chembl_id[0]
selected_target

'CHEMBL614818'

In [11]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="GI50")

In [12]:
df = pd.DataFrame.from_dict(res)
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,2388082,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,5.6
1,None,2388083,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1.7
2,None,2388084,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,10.0
3,None,2388085,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,2.8
4,None,2388086,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,None,23257174,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,20.01
128,None,23257175,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9.09
129,None,24415209,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9.607999999999999
130,None,24415210,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,11.380999999999998


In [ ]:
df = pd.read_csv('/content/bioactivity_data_UO-31.csv', header = 0)

<ipython-input-176-0c86e62881e9>:1: DtypeWarning: Columns (15,25,28,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/bioactivity_data_UO-31.csv', header = 0)


In [13]:
# considering nM unit for MIC
#df = df[(df['standard_type'] == 'GI50')]
#df = df[(df['standard_type'] == 'IC50')]
#df = df[(df['standard_type'] == 'EC50')]
df = df[(df['standard_units'] == 'nM')]
df = df[(df['relation'] == '=')]
df = df[df.canonical_smiles.notna()]
df = df[df.standard_value.notna()]
df

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,2388082,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,5.6
1,None,2388083,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1.7
3,None,2388085,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,2.8
4,None,2388086,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1.5
5,None,2388087,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,None,23257174,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,20.01
128,None,23257175,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9.09
129,None,24415209,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9.607999999999999
130,None,24415210,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,Homo sapiens,HEK293,9606,None,None,GI50,uM,UO_0000065,None,11.380999999999998


In [14]:
fix = []

for i in df.standard_value:
    fix.append(float(i))

STATUS = []

for i in fix:
    if i < 1000:
        STATUS.append("Yes") #active
        
    elif 20000 <= i <= 100000:
        STATUS.append("No") #inactive
        
    else:
        STATUS.append("IN") #intermediate     

df = df.drop('value', 1)
df['value'] = fix
df['Activity'] = STATUS
df

<ipython-input-14-d453deb383ef>:18: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('value', 1)


,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value,Activity
0,None,2388082,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,5600.0,IN
1,None,2388083,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1700.0,IN
3,None,2388085,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,2800.0,IN
4,None,2388086,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,1500.0,IN
5,None,2388087,[],CHEMBL1001102,Cytotoxicity against HEK293 cells after 48 hrs...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,700.0,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,None,23257174,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,20010.0,No
128,None,23257175,[],CHEMBL4832360,Cytotoxicity against HEK293 cells assessed as ...,A,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9090.0,IN
129,None,24415209,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,9608.0,IN
130,None,24415210,"[{'comments': None, 'relation': '=', 'result_f...",CHEMBL5047217,Antiproliferative activity against human HEK29...,T,None,None,BAO_0000189,BAO_0000219,...,HEK293,9606,None,None,GI50,uM,UO_0000065,None,11381.0,IN


In [15]:
# considering nM unit for MIC
df = df[(df['Activity'] != 'IN')]
len(df)

37

In [16]:
df.to_csv('/content/bioactivity_data_HEK293_GI50.csv', index=False)
     